##### Hello, MNIST

In [27]:
# import mnist dataset from sklearn
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
x, y = mnist['data'], mnist['target']
x.shape, y.shape

((70000, 784), (70000,))

In [28]:
print(type(x))
print(type(y))
print(y.dtypes)
print([type(i) for i in y[:1]])

# since y' dtype is str, we need to convert it to float
y = y.astype(float)
print([type(i) for i in y[:1]])

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
category
[<class 'str'>]
[<class 'float'>]


In [29]:
# normalize x to keep our gradients manageable
x = x / 255

In [31]:
# to build a zero-classifier, make the label 1 when we have a zero, and 0 otherwise
import numpy as np
y_new = np.zeros(shape=y.shape)
y_new[np.where(y == 0.0)] = 1
y = y_new

In [35]:
# we can make our train/test split. we'll also transform the data into the shape we want, with each example in a column(instead of a row)
m = 60000
m_test = x.shape[0] - m

x_train, x_test = x[:m].T, x[m:].T
y_train, y_test = y[:m].reshape(1,m), y[m:].reshape(1,m_test)

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(784, 60000) (1, 60000) (784, 10000) (1, 10000)


In [36]:
# finally we shuffle the training set for good measure
np.random.seed(138)
shuffle_index = np.random.permutation(m)
x_train, y_train = x_train[:, shuffle_index], y_train[:, shuffle_index]
print(x_train.shape, y_test.shape)

InvalidIndexError: (slice(None, None, None), array([45084, 33389, 21425, ..., 33653, 57833, 58618]))

In [38]:
np.unique(y, return_counts=True)

(array([0., 1.]), array([63097,  6903], dtype=int64))